In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import heapq
from collections import defaultdict

# Step 2: Calculate frequency of each coefficient
def calculate_frequency(data):
    frequency = defaultdict(int)
    for value in data.ravel():
        frequency[value] += 1
    return frequency

# Step 3: Huffman Tree and Encoding (reuse from your code)
class HuffmanNode:
    def __init__(self, value, frequency):
        self.value = value
        self.frequency = frequency
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.frequency < other.frequency

def build_huffman_tree(frequency):
    priority_queue = [HuffmanNode(value, freq) for value, freq in frequency.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        
        internal_node = HuffmanNode(None, left.frequency + right.frequency)
        internal_node.left = left
        internal_node.right = right
        
        heapq.heappush(priority_queue, internal_node)

    return priority_queue[0]  # Root of the tree

def generate_huffman_codes(node, current_code='', codes=defaultdict(str)):
    if node is not None:
        if node.value is not None:
            codes[node.value] = current_code
        generate_huffman_codes(node.left, current_code + '0', codes)
        generate_huffman_codes(node.right, current_code + '1', codes)
    return codes

def compress_data(data, huffman_codes):
    compressed_data = ''.join(huffman_codes[value] for value in data.ravel())
    return compressed_data

def decompress_data(compressed_data, huffman_tree, original_shape):
    current_node = huffman_tree
    decompressed_data = []

    for bit in compressed_data:
        current_node = current_node.left if bit == '0' else current_node.right
        if current_node.value is not None:
            decompressed_data.append(current_node.value)
            current_node = huffman_tree

    return np.array(decompressed_data).reshape(original_shape)

# Step 4: Apply Transform Coding
def apply_transform(image):
    # Apply 2D DCT on blocks of the image
    block_size = 8
    transformed_image = np.zeros_like(image, dtype=float)

    for i in range(0, image.shape[0], block_size):
        for j in range(0, image.shape[1], block_size):
            block = image[i:i+block_size, j:j+block_size]
            dct_block = cv2.dct(np.float32(block))
            transformed_image[i:i+block_size, j:j+block_size] = dct_block

    return transformed_image

# Step 5: Quantize the DCT coefficients
# Step 5: Quantize the DCT coefficients block-wise
def quantize_dct(dct_image, quantization_matrix):
    block_size = quantization_matrix.shape[0]
    quantized_image = np.zeros_like(dct_image, dtype=int)

    for i in range(0, dct_image.shape[0], block_size):
        for j in range(0, dct_image.shape[1], block_size):
            block = dct_image[i:i+block_size, j:j+block_size]
            quantized_image[i:i+block_size, j:j+block_size] = np.round(block / quantization_matrix)
    
    return quantized_image

# Step 6: Dequantize the DCT coefficients block-wise
def dequantize_dct(quantized_image, quantization_matrix):
    block_size = quantization_matrix.shape[0]
    dequantized_image = np.zeros_like(quantized_image, dtype=float)

    for i in range(0, quantized_image.shape[0], block_size):
        for j in range(0, quantized_image.shape[1], block_size):
            block = quantized_image[i:i+block_size, j:j+block_size]
            dequantized_image[i:i+block_size, j:j+block_size] = block * quantization_matrix
    
    return dequantized_image

# Step 6: Apply inverse DCT to reconstruct the image
def apply_inverse_transform(dct_image):
    block_size = 8
    reconstructed_image = np.zeros_like(dct_image, dtype=float)

    for i in range(0, dct_image.shape[0], block_size):
        for j in range(0, dct_image.shape[1], block_size):
            block = dct_image[i:i+block_size, j:j+block_size]
            idct_block = cv2.idct(np.float32(block))
            reconstructed_image[i:i+block_size, j:j+block_size] = idct_block

    return np.clip(reconstructed_image, 0, 255).astype(np.uint8)

# Step 7: Process the Image
def process_transform(image_path, quantization_matrix):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply DCT
    transformed_image = apply_transform(image)
    
    # Quantize DCT coefficients
    quantized_image = quantize_dct(transformed_image, quantization_matrix)
    
    # Flatten and calculate frequency
    frequency = calculate_frequency(quantized_image)
    
    # Build Huffman Tree and generate codes
    huffman_tree = build_huffman_tree(frequency)
    huffman_codes = generate_huffman_codes(huffman_tree)
    
    # Compress
    compressed_data = compress_data(quantized_image, huffman_codes)
    
    # Decompress and reconstruct
    decompressed_quantized_image = decompress_data(compressed_data, huffman_tree, quantized_image.shape)
    dequantized_image = dequantize_dct(decompressed_quantized_image, quantization_matrix)
    reconstructed_image = apply_inverse_transform(dequantized_image)
    
    return compressed_data, reconstructed_image

# Step 8: Define Quantization Matrix (JPEG example)
quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])
def calculate_metrics(original_image1, reconstructed_image1, bitstream1,
                      original_image2, reconstructed_image2, bitstream2):
    # Calculate MSE for both images
    mse1 = np.mean((original_image1 - reconstructed_image1) ** 2)
    mse2 = np.mean((original_image2 - reconstructed_image2) ** 2)
    SUM = mse1 + mse2
    
    # Calculate compression ratio (ρ)
    total_bits_original = 2 * 1200 * 1200 * 8
    total_bits_compressed = len(bitstream1)+len(bitstream2)
    rho = total_bits_compressed / total_bits_original
    
    # Combined score
    score = SUM + (200 * rho)
    
    # print(f"Compressed Bits (Image1): {sum(len(chunk) for chunk in bitstream1)}")
    # print(f"Compressed Bits (Image2): {sum(len(chunk) for chunk in bitstream2)}")
    # print(f"Original Bits: {total_bits_original}")
    print(f"Compression Ratio (ρ): {rho}")
    print(f"MSE1:{mse1},MSE2:{mse2}")
    print(f"SCORE:{score}")
    return SUM, rho, score
# Process images
c1, r1 = process_transform('Watermarked_Image1.tiff', quantization_matrix)
c2, r2 = process_transform('Watermarked_Image2.tiff', quantization_matrix)

# Original images
og1 = cv2.imread('Watermarked_Image1.tiff', cv2.IMREAD_GRAYSCALE)
og2 = cv2.imread('Watermarked_Image2.tiff', cv2.IMREAD_GRAYSCALE)

# Metrics
calculate_metrics(og1, r1, c1, og2, r2, c2)


Compression Ratio (ρ): 0.1990642361111111
MSE1:5.092339583333334,MSE2:35.81712777777778
SCORE:80.72231458333333


(np.float64(40.90946736111111),
 0.1990642361111111,
 np.float64(80.72231458333333))

In [5]:
import pandas as pd
def group(compressed_data):
    byte_data = []
    for i in range(0, len(compressed_data), 8):
        byte_data.append(compressed_data[i:i+8])
    return byte_data

bitstreamimage1=group(c1)
bitstreamimage2=group(c2)
size1=1200*1200-len(bitstreamimage1)
size2=1200*1200-len(bitstreamimage2)


submission = pd.DataFrame({
"ID": list(range(1200*1200)),
"CompressedImage1": np.array(bitstreamimage1+(["I"]*size1)),
    "CompressedImage2":  np.array(bitstreamimage2+(["I"]*size2)),
    "Image1": np.array(r1).flatten(),
    "Image2": np.array(r2).flatten(),} )

submission.to_csv("submission.csv", index=False)

